<a href="https://colab.research.google.com/github/DamuluruSanjana/Inconsistency/blob/main/INCONSISTENCY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CONNECTION


In [1]:
# Step 1: Download Spark
!wget https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

# Step 2: Extract Spark
!ls -l
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!ls -l

# Step 3: Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

# Step 4: Initialize findspark
!pip install -q findspark
import findspark
findspark.init()
print("SPARK_HOME is set to:", os.environ["SPARK_HOME"])

# Step 5: Verify Spark Installation
!pip install -q pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("TestSpark").getOrCreate()
print("Spark session created successfully!")
spark

--2025-03-19 16:55:43--  https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228834641 (218M) [application/x-gzip]
Saving to: ‘spark-3.1.2-bin-hadoop3.2.tgz’

spark-3.1.2-bin-had 100%[===================>] 218.23M  15.1MB/s    in 62s     

2025-03-19 16:56:46 (3.50 MB/s) - ‘spark-3.1.2-bin-hadoop3.2.tgz’ saved [228834641/228834641]

total 223480
drwxr-xr-x 1 root root      4096 Mar 17 13:32 sample_data
-rw-r--r-- 1 root root 228834641 May 24  2021 spark-3.1.2-bin-hadoop3.2.tgz
total 223484
drwxr-xr-x  1 root root      4096 Mar 17 13:32 sample_data
drwxr-xr-x 13 1000 1000      4096 May 24  2021 spark-3.1.2-bin-hadoop3.2
-rw-r--r--  1 root root 228834641 May 24  2021 spark-3.1.2-bin-hadoop3.2.tgz
SPARK_HOME is set to: /content/spark

Exception: Java gateway process exited before sending its port number

In [2]:
# Mount Google Drive to access your datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


SAMPLE FILE

In [3]:
import pandas as pd

# Create sample data with an 'id' column and a 'text_column'
data_dict = {
    'id': [1, 2, 3, 4, 5],
    'text_column': [
        "This is a sample text.",
        "This text is another example.",
        "Sample text with duplicate sample text.",
        "Another row with unique content.",
        "This text contains some noise, e.g., punctuation!"
    ]
}

# Create a DataFrame from the sample data
df = pd.DataFrame(data_dict)

# Save the DataFrame to a CSV file named 'sample.csv' in the current directory
csv_file_path = 'sample.csv'
df.to_csv(csv_file_path, index=False)

# Display the DataFrame and confirm the CSV file generation
print("Sample CSV generated successfully:")
print(df)

Sample CSV generated successfully:
   id                                        text_column
0   1                             This is a sample text.
1   2                      This text is another example.
2   3            Sample text with duplicate sample text.
3   4                   Another row with unique content.
4   5  This text contains some noise, e.g., punctuation!


DATA PROCESSING

In [4]:
# Import pandas to work with CSV files
import pandas as pd

# Set the path to your CSV file stored in Google Drive
csv_file_path = '/content/sample.csv'  # update this path if needed

# Read the CSV file into a DataFrame
data = pd.read_csv(csv_file_path)

# Display the first few rows to verify the data
print("Sample data from CSV:")
print(data.head())

Sample data from CSV:
   id                                        text_column
0   1                             This is a sample text.
1   2                      This text is another example.
2   3            Sample text with duplicate sample text.
3   4                   Another row with unique content.
4   5  This text contains some noise, e.g., punctuation!


In [5]:
import nltk
# Download the missing punkt_tab resource
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
# Import nltk and download the tokenizer models
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

# Define a function to clean and tokenize text
def clean_and_tokenize(text):
    # Convert to lowercase and strip leading/trailing whitespace
    text = text.lower().strip()
    # Tokenize the text into words
    tokens = word_tokenize(text)
    return tokens

# Assuming the CSV has a column named 'text_column'
# Apply the cleaning and tokenization to create a new 'tokens' column
data['tokens'] = data['text_column'].apply(clean_and_tokenize)

# Display the updated DataFrame with the new 'tokens' column
print("Data after tokenization:")
print(data.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Data after tokenization:
   id                                        text_column  \
0   1                             This is a sample text.   
1   2                      This text is another example.   
2   3            Sample text with duplicate sample text.   
3   4                   Another row with unique content.   
4   5  This text contains some noise, e.g., punctuation!   

                                              tokens  
0                     [this, is, a, sample, text, .]  
1              [this, text, is, another, example, .]  
2   [sample, text, with, duplicate, sample, text, .]  
3           [another, row, with, unique, content, .]  
4  [this, text, contains, some, noise, ,, e.g., ,...  


In [7]:
import pandas as pd
import re

# Load sample CSV file (make sure sample.csv exists in the current directory)
data = pd.read_csv('sample.csv')
print("Original Data:")
print(data, "\n")

# -------------------------------
# 1. Duplicate Content Detection
# -------------------------------
def detect_duplicates(df, text_column='text_column'):
    """
    Returns a DataFrame with rows that have duplicate values in the specified text column.
    """
    duplicates = df[df.duplicated(subset=[text_column], keep=False)]
    return duplicates

duplicates_df = detect_duplicates(data)

# -------------------------------
# 2. Formatting Errors Detection
# -------------------------------
def detect_formatting_issues(text):
    """
    Checks whether the text:
    - Starts with an uppercase letter.
    - Ends with proper punctuation (., !, or ?).

    Returns a list of formatting issues.
    """
    issues = []
    if not text:
        issues.append("Empty text")
        return issues
    if not text[0].isupper():
        issues.append("Does not start with an uppercase letter")
    if text[-1] not in ['.', '!', '?']:
        issues.append("Does not end with proper punctuation")
    return issues

# -------------------------------
# 3. Conflicting Information Detection
# -------------------------------
def detect_conflicting_information(text):
    """
    Checks for pairs of contradictory keywords in the text.
    Example conflict pairs include:
      - ('increase', 'decrease')
      - ('up', 'down')
      - ('true', 'false')
      - ('positive', 'negative')
      - ('win', 'loss')
      - ('hot', 'cold')

    Returns a list of detected conflicts.
    """
    conflict_pairs = [
        ('increase', 'decrease'),
        ('up', 'down'),
        ('true', 'false'),
        ('positive', 'negative'),
        ('win', 'loss'),
        ('hot', 'cold')
    ]
    conflicts = []
    text_lower = text.lower()
    for w1, w2 in conflict_pairs:
        if w1 in text_lower and w2 in text_lower:
            conflicts.append(f"Conflict: {w1} vs {w2}")
    return conflicts

# -------------------------------
# 4. Data Anomalies Detection
# -------------------------------
def detect_data_anomalies(text):
    """
    Detects data anomalies such as:
    - Empty text.
    - Text that is extremely short.
    - Excessive punctuation (three or more punctuation marks in a row).
    - A high ratio of non-alphanumeric characters compared to alphanumeric ones.

    Returns a list of anomaly issues.
    """
    anomalies = []
    if not text or len(text.strip()) == 0:
        anomalies.append("Empty text anomaly")
    if len(text) < 5:
        anomalies.append("Text too short")
    # Detect excessive punctuation (3+ consecutive punctuation marks)
    if re.search(r'[\!\?\.]{3,}', text):
        anomalies.append("Excessive punctuation")
    # Check for a high ratio of non-alphanumeric to alphanumeric characters
    alpha_numeric = re.findall(r'[a-zA-Z0-9]', text)
    non_alphanumeric = re.findall(r'[^a-zA-Z0-9\s]', text)
    if alpha_numeric and (len(non_alphanumeric) / len(alpha_numeric)) > 0.5:
        anomalies.append("High non-alphanumeric ratio")
    return anomalies

# -------------------------------
# Apply All Detection Functions
# -------------------------------
# Apply formatting, conflicting information, and data anomalies detectors
data['formatting_issues'] = data['text_column'].apply(detect_formatting_issues)
data['conflicting_information'] = data['text_column'].apply(detect_conflicting_information)
data['data_anomalies'] = data['text_column'].apply(detect_data_anomalies)

# -------------------------------
# Output the Results
# -------------------------------
print("Detected Duplicate Entries:")
print(duplicates_df, "\n")

print("Data with Inconsistency Issues Detected:")
print(data[['id', 'text_column', 'formatting_issues', 'conflicting_information', 'data_anomalies']])

Original Data:
   id                                        text_column
0   1                             This is a sample text.
1   2                      This text is another example.
2   3            Sample text with duplicate sample text.
3   4                   Another row with unique content.
4   5  This text contains some noise, e.g., punctuation! 

Detected Duplicate Entries:
Empty DataFrame
Columns: [id, text_column]
Index: [] 

Data with Inconsistency Issues Detected:
   id                                        text_column formatting_issues  \
0   1                             This is a sample text.                []   
1   2                      This text is another example.                []   
2   3            Sample text with duplicate sample text.                []   
3   4                   Another row with unique content.                []   
4   5  This text contains some noise, e.g., punctuation!                []   

  conflicting_information data_anomalies  
0    

TESTING WITH CSV FILE

In [9]:
import pandas as pd
import re
import io
import ipywidgets as widgets
from IPython.display import display, FileLink, clear_output

# -------------------------------
# Inconsistency Detection Functions
# -------------------------------

def detect_formatting_issues(text):
    """
    Checks whether the text:
      - Starts with an uppercase letter.
      - Ends with proper punctuation (., !, or ?).
    Returns a list of formatting issues.
    """
    issues = []
    if not text:
        issues.append("Empty text")
        return issues
    if not text[0].isupper():
        issues.append("Does not start with an uppercase letter")
    if text[-1] not in ['.', '!', '?']:
        issues.append("Does not end with proper punctuation")
    return issues

def detect_conflicting_information(text):
    """
    Checks for pairs of contradictory keywords in the text.
    Returns a list of detected conflicts.
    """
    conflict_pairs = [
        ('increase', 'decrease'),
        ('up', 'down'),
        ('true', 'false'),
        ('positive', 'negative'),
        ('win', 'loss'),
        ('hot', 'cold')
    ]
    conflicts = []
    text_lower = text.lower()
    for w1, w2 in conflict_pairs:
        if w1 in text_lower and w2 in text_lower:
            conflicts.append(f"Conflict: {w1} vs {w2}")
    return conflicts

def detect_data_anomalies(text):
    """
    Detects data anomalies such as:
      - Empty text.
      - Text that is extremely short.
      - Excessive punctuation (three or more punctuation marks in a row).
      - A high ratio of non-alphanumeric to alphanumeric characters.
    Returns a list of anomaly issues.
    """
    anomalies = []
    if not text or len(text.strip()) == 0:
        anomalies.append("Empty text anomaly")
    if len(text) < 5:
        anomalies.append("Text too short")
    if re.search(r'[\!\?\.]{3,}', text):
        anomalies.append("Excessive punctuation")
    alpha_numeric = re.findall(r'[a-zA-Z0-9]', text)
    non_alphanumeric = re.findall(r'[^a-zA-Z0-9\s]', text)
    if alpha_numeric and (len(non_alphanumeric) / len(alpha_numeric)) > 0.5:
        anomalies.append("High non-alphanumeric ratio")
    return anomalies

def detect_duplicates(df, text_column='text_column'):
    """
    Returns a DataFrame with rows that have duplicate values in the specified text column.
    """
    duplicates = df[df.duplicated(subset=[text_column], keep=False)]
    return duplicates

def correct_text(text):
    """
    Corrects formatting issues:
      - Capitalizes the first character if necessary.
      - Ensures the text ends with a punctuation mark (adds a period if missing).
    Returns the corrected text.
    """
    if not text:
        return text
    # Capitalize first letter if not already
    if not text[0].isupper():
        text = text[0].upper() + text[1:]
    # Append period if missing ending punctuation
    if text[-1] not in ['.', '!', '?']:
        text = text + '.'
    return text

# -------------------------------
# Build the Upload & Processing Interface
# -------------------------------

# Create a file upload widget (accepting only CSV files)
upload_widget = widgets.FileUpload(accept='.csv', multiple=False)

# Create a process button widget
process_button = widgets.Button(description="Process Document")

# Output area to display results and download link
output_area = widgets.Output()

def process_file(button):
    with output_area:
        clear_output()  # Clear previous output
        if upload_widget.value:
            # Get the uploaded file (only one file allowed)
            uploaded_filename = list(upload_widget.value.keys())[0]
            content = upload_widget.value[uploaded_filename]['content']
            # Read CSV from the uploaded file content
            df = pd.read_csv(io.BytesIO(content))

            print("Original Data:")
            print(df.head(), "\n")

            # Apply inconsistency detection on the 'text_column'
            df['formatting_issues'] = df['text_column'].apply(detect_formatting_issues)
            df['conflicting_information'] = df['text_column'].apply(detect_conflicting_information)
            df['data_anomalies'] = df['text_column'].apply(detect_data_anomalies)

            # Check for duplicate content
            duplicates_df = detect_duplicates(df)
            print("Detected Duplicate Entries:")
            if duplicates_df.empty:
                print("No duplicate entries found.\n")
            else:
                print(duplicates_df, "\n")

            # Create a corrected text column for basic formatting corrections
            df['corrected_text'] = df['text_column'].apply(correct_text)

            print("Processed Data with Inconsistency Report:")
            display(df[['text_column', 'formatting_issues', 'conflicting_information', 'data_anomalies', 'corrected_text']])

            # Save the corrected document to CSV
            corrected_filename = "corrected_document.csv"
            df.to_csv(corrected_filename, index=False)
            print("\nDownload the corrected document:")
            display(FileLink(corrected_filename))
        else:
            print("No file uploaded. Please upload a CSV file.")

# Bind the process_file function to the button click event
process_button.on_click(process_file)

# Display the interface: file upload widget, process button, and output area
display(widgets.VBox([upload_widget, process_button, output_area]))

In [10]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.0 MB/s eta 0:00:00


In [11]:
import os
import json
import pandas as pd
import re
import gradio as gr
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

# -------------------------------
# Inconsistency Detection Functions
# -------------------------------

def detect_formatting_issues(text):
    """
    Checks whether the text:
      - Starts with an uppercase letter.
      - Ends with proper punctuation (., !, or ?).
    Returns a list of formatting issues.
    """
    issues = []
    if not text:
        issues.append("Empty text")
        return issues
    if not text[0].isupper():
        issues.append("Does not start with an uppercase letter")
    if text[-1] not in ['.', '!', '?']:
        issues.append("Does not end with proper punctuation")
    return issues

def detect_conflicting_information(text):
    """
    Checks for pairs of contradictory keywords in the text.
    Returns a list of detected conflicts.
    """
    conflict_pairs = [
        ('increase', 'decrease'),
        ('up', 'down'),
        ('true', 'false'),
        ('positive', 'negative'),
        ('win', 'loss'),
        ('hot', 'cold')
    ]
    conflicts = []
    text_lower = text.lower()
    for w1, w2 in conflict_pairs:
        if w1 in text_lower and w2 in text_lower:
            conflicts.append(f"Conflict: {w1} vs {w2}")
    return conflicts

def detect_data_anomalies(text):
    """
    Detects data anomalies such as:
      - Empty text.
      - Text that is extremely short.
      - Excessive punctuation (three or more punctuation marks in a row).
      - A high ratio of non-alphanumeric to alphanumeric characters.
    Returns a list of anomaly issues.
    """
    anomalies = []
    if not text or len(text.strip()) == 0:
        anomalies.append("Empty text anomaly")
    if len(text) < 5:
        anomalies.append("Text too short")
    if re.search(r'[\!\?\.]{3,}', text):
        anomalies.append("Excessive punctuation")
    alpha_numeric = re.findall(r'[a-zA-Z0-9]', text)
    non_alphanumeric = re.findall(r'[^a-zA-Z0-9\s]', text)
    if alpha_numeric and (len(non_alphanumeric) / len(alpha_numeric)) > 0.5:
        anomalies.append("High non-alphanumeric ratio")
    return anomalies

def detect_duplicates(df, text_column='text_column'):
    """
    Returns a DataFrame with rows that have duplicate values in the specified text column.
    """
    duplicates = df[df.duplicated(subset=[text_column], keep=False)]
    return duplicates

def correct_text(text):
    """
    Corrects formatting issues:
      - Capitalizes the first character if necessary.
      - Ensures the text ends with a punctuation mark (adds a period if missing).
    Returns the corrected text.
    """
    if not text:
        return text
    if not text[0].isupper():
        text = text[0].upper() + text[1:]
    if text[-1] not in ['.', '!', '?']:
        text = text + '.'
    return text

# -------------------------------
# Hadoop Integration Function
# -------------------------------
def upload_to_hdfs(local_file, hdfs_path):
    """
    Uploads a local file to HDFS using a shell command.
    Requires Hadoop to be installed and HDFS running.
    """
    command = f"hdfs dfs -put -f {local_file} {hdfs_path}"
    os.system(command)
    return f"File uploaded to HDFS at: {hdfs_path}"

# -------------------------------
# Gradio Interface Processing Function with Spark & Hadoop
# -------------------------------

def process_file(file_obj):
    if file_obj is None:
        return pd.DataFrame(), pd.DataFrame(), None, "", None

    ext = os.path.splitext(file_obj.name)[1].lower()

    # Initialize Spark session
    spark = SparkSession.builder.master("local[*]").appName("InconsistencyDetection").getOrCreate()

    try:
        if ext == ".csv":
            spark_df = spark.read.option("header", "true").csv(file_obj.name)
        elif ext in [".txt"]:
            spark_df = spark.read.text(file_obj.name)
            spark_df = spark_df.withColumnRenamed("value", "text_column")
        elif ext in [".json"]:
            try:
                spark_df = spark.read.json(file_obj.name)
                if "text_column" not in spark_df.columns:
                    with open(file_obj.name, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    if isinstance(data, list) and all(isinstance(x, str) for x in data):
                        spark_df = spark.createDataFrame([(x,) for x in data], ["text_column"])
                    else:
                        return pd.DataFrame({"Error": ["JSON file does not contain a 'text_column'."]}), pd.DataFrame(), None, "", None
            except Exception as e:
                return pd.DataFrame({"Error": [f"Error reading JSON file: {e}"]}), pd.DataFrame(), None, "", None
        elif ext in [".xml"]:
            try:
                df = pd.read_xml(file_obj.name)
                if 'text_column' not in df.columns:
                    return pd.DataFrame({"Error": ["XML file does not contain a 'text_column'."]}), pd.DataFrame(), None, "", None
                spark_df = spark.createDataFrame(df)
            except Exception as e:
                return pd.DataFrame({"Error": [f"Error reading XML file: {e}"]}), pd.DataFrame(), None, "", None
        else:
            return pd.DataFrame({"Error": ["Unsupported file format. Supported formats: CSV, TXT, JSON, XML."]}), pd.DataFrame(), None, "", None
    except Exception as e:
        return pd.DataFrame({"Error": [f"Error reading file: {e}"]}), pd.DataFrame(), None, "", None

    # Define Spark UDFs wrapping our Python functions
    detect_formatting_udf = udf(lambda x: detect_formatting_issues(x), ArrayType(StringType()))
    detect_conflict_udf = udf(lambda x: detect_conflicting_information(x), ArrayType(StringType()))
    detect_anomalies_udf = udf(lambda x: detect_data_anomalies(x), ArrayType(StringType()))
    correct_text_udf = udf(lambda x: correct_text(x), StringType())

    # Apply UDFs to create new columns
    spark_df = spark_df.withColumn("formatting_issues", detect_formatting_udf(spark_df["text_column"]))
    spark_df = spark_df.withColumn("conflicting_information", detect_conflict_udf(spark_df["text_column"]))
    spark_df = spark_df.withColumn("data_anomalies", detect_anomalies_udf(spark_df["text_column"]))
    spark_df = spark_df.withColumn("corrected_text", correct_text_udf(spark_df["text_column"]))

    # Convert the Spark DataFrame to a Pandas DataFrame for further processing
    pdf = spark_df.toPandas()

    # Duplicate detection using our pandas function
    duplicates_df = detect_duplicates(pdf)
    if duplicates_df.empty:
        duplicate_report = pd.DataFrame({"Message": ["No duplicate entries found."]})
    else:
        duplicate_report = duplicates_df.copy()

    processed_preview = pdf[['text_column', 'formatting_issues',
                             'conflicting_information', 'data_anomalies',
                             'corrected_text']]

    # Save the corrected DataFrame as a downloadable CSV file
    corrected_filename = "corrected_document.csv"
    pdf.to_csv(corrected_filename, index=False)

    # -------------------------------
    # Hadoop: Upload Corrected File to HDFS
    # -------------------------------
    # Adjust the HDFS path based on your Hadoop configuration
    hdfs_destination = "/user/your_username/corrected_document.csv"
    hadoop_status = upload_to_hdfs(corrected_filename, hdfs_destination)

    # -------------------------------
    # Analysis & Graph Generation
    # -------------------------------
    total_records = len(pdf)
    records_with_errors = pdf.apply(lambda row: len(row['formatting_issues'] + row['conflicting_information'] + row['data_anomalies']) > 0, axis=1).sum()

    issues_counts = {}
    for idx, row in pdf.iterrows():
        issues = row['formatting_issues'] + row['conflicting_information'] + row['data_anomalies']
        for issue in issues:
            issues_counts[issue] = issues_counts.get(issue, 0) + 1

    analysis_text = f"Analysis Summary:\nTotal records: {total_records}\n"
    analysis_text += f"Records with at least one inconsistency: {records_with_errors}\n"
    analysis_text += "Breakdown of inconsistency types:\n"
    for issue, count in issues_counts.items():
        analysis_text += f" - {issue}: {count} occurrence(s)\n"
    analysis_text += f"\nHadoop Upload Status: {hadoop_status}"

    fig, ax = plt.subplots(figsize=(8, 4))
    if issues_counts:
        x_vals = list(issues_counts.keys())
        y_vals = list(issues_counts.values())
        ax.plot(x_vals, y_vals, marker='o', linestyle='-', color='teal')
        ax.set_title("Trend of Detected Inconsistencies")
        ax.set_xlabel("Inconsistency Type")
        ax.set_ylabel("Count")
        plt.xticks(rotation=45, ha="right")
    else:
        ax.text(0.5, 0.5, 'No inconsistencies detected', horizontalalignment='center',
                verticalalignment='center', transform=ax.transAxes, fontsize=12)
        ax.set_axis_off()
    plt.tight_layout()

    return processed_preview, duplicate_report, corrected_filename, analysis_text, fig

# -------------------------------
# Custom CSS for Black Background Interface
# -------------------------------
custom_css = """
body {
  background-color: #000;
  color: #fff;
}
.container {
  border: 1px solid #444;
  border-radius: 10px;
  padding: 20px;
  background-color: #222;
  margin: 10px;
}
h1, h2, h3, h4, h5, h6 {
  color: #fff;
  text-align: center;
}
"""

# -------------------------------
# Build and Launch the Gradio Blocks Interface
# -------------------------------
with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("<h1>CSV Inconsistency Detection & Correction with Spark & Hadoop</h1>")
    gr.Markdown(
        "Upload a file containing a column named **'text_column'**. "
        "This tool uses Apache Spark for data processing and Hadoop for file storage. "
        "It detects formatting issues, conflicting information, and data anomalies, "
        "and then produces a corrected file for download. "
        "Supported formats: CSV, TXT, JSON, XML."
    )

    with gr.Row():
        with gr.Column(elem_classes="container"):
            file_input = gr.File(label="Upload File", file_count="single", file_types=['.csv', '.txt', '.json', '.xml'])
            process_button = gr.Button("Process Document")
        with gr.Column(elem_classes="container"):
            # This widget provides the download link for the corrected CSV file.
            download_csv = gr.File(label="Download Corrected CSV")

    with gr.Row():
        with gr.Column(elem_classes="container"):
            processed_preview = gr.Dataframe(label="Processed Data Preview")
        with gr.Column(elem_classes="container"):
            duplicate_report = gr.Dataframe(label="Duplicate Report")

    with gr.Row():
        with gr.Column(elem_classes="container"):
            analysis_summary = gr.Textbox(label="Analysis Summary", lines=8)
        with gr.Column(elem_classes="container"):
            plot_output = gr.Plot(label="Inconsistencies Trend Graph")

    process_button.click(
        fn=process_file,
        inputs=file_input,
        outputs=[processed_preview, duplicate_report, download_csv, analysis_summary, plot_output]
    )

demo.launch(share=True, inline=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://32983b0c7ba9ffa82a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Code to Generate Document with Inconsistencies


In [12]:
import pandas as pd

# Create a DataFrame with intentional inconsistencies
data = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'text_column': [
        "this is a sample text",                         # Formatting: not capitalized, missing punctuation.
        "This text is another example",                   # Formatting: missing ending punctuation.
        "Increase profits, decrease costs.",              # Conflicting info: "increase" and "decrease".
        "Short",                                          # Data anomaly: too short, might be considered too brief.
        "Unique content!!!",                              # Formatting anomaly: excessive punctuation.
        "this text contains noise, e.g., punctuation",    # Formatting: not capitalized, missing punctuation.
        "Sample text with duplicate sample text.",        # Duplicate candidate.
        "Sample text with duplicate sample text.",        # Duplicate candidate.
        "up and down",                                    # Conflicting info: "up" and "down".
        "win loss"                                        # Conflicting info: "win" and "loss", missing punctuation.
    ]
}

# Create a DataFrame from the dictionary
df_inconsistent = pd.DataFrame(data)

# Save the DataFrame to a CSV file
csv_filename = "inconsistent_sample.csv"
df_inconsistent.to_csv(csv_filename, index=False)

# Display the DataFrame and confirm file creation
print("Generated CSV with inconsistencies:")
print(df_inconsistent)

Generated CSV with inconsistencies:
   id                                  text_column
0   1                        this is a sample text
1   2                 This text is another example
2   3            Increase profits, decrease costs.
3   4                                        Short
4   5                            Unique content!!!
5   6  this text contains noise, e.g., punctuation
6   7      Sample text with duplicate sample text.
7   8      Sample text with duplicate sample text.
8   9                                  up and down
9  10                                     win loss
